#Importação dos pacotes necessários e etapas iniciais

In [19]:
# Importação dos pacotes
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession
import gradio as gr
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator


In [2]:
# Cria a SparkSession - usada quando se trabalha com Dataframes no Spark
spark = SparkSession.builder \
    .appName("ALS Example") \
    .master("local[*]") \
    .getOrCreate()

# Obtém o SparkContext a partir da SparkSession
sc = spark.sparkContext  # Cria o SparkContext, que gerencia os recursos e executa operações com RDDs

# Agora você pode carregar o arquivo com sc.textFile
ratingsRDD = sc.textFile("user-item.txt")  # Carrega o arquivo como um RDD, cada linha é um elemento
ratingsRDD.collect()  # Retorna todos os elementos do RDD como uma lista (útil para inspeção em pequenos conjuntos)


['1001,9001,10',
 '1001,9002,1',
 '1001,9003,9',
 '1002,9001,3',
 '1002,9002,5',
 '1002,9003,1',
 '1002,9004,10',
 '1003,9001,2',
 '1003,9002,6',
 '1003,9003,2',
 '1003,9004,9',
 '1003,9005,10',
 '1003,9006,8',
 '1003,9007,9',
 '1004,9001,9',
 '1004,9002,2',
 '1004,9003,8',
 '1004,9004,3',
 '1004,9010,10',
 '1004,9011,9',
 '1004,9012,8',
 '1005,9001,8',
 '1005,9002,3',
 '1005,9003,7',
 '1005,9004,1',
 '1005,9010,9',
 '1005,9011,10',
 '1005,9012,9',
 '1005,9013,8',
 '1005,9014,1',
 '1005,9015,1',
 '1006,9001,7',
 '1006,9002,4',
 '1006,9003,8',
 '1006,9004,1',
 '1006,9010,7',
 '1006,9011,6',
 '1006,9012,9',
 '1007,11853,5',
 '1007,10097,6',
 '1007,9194,5',
 '1007,13021,7',
 '1007,12392,3',
 '1007,10560,4',
 '1007,10124,9',
 '1007,12717,4',
 '1007,11861,7',
 '1008,9768,9',
 '1008,11908,5',
 '1008,10949,8',
 '1008,11572,3',
 '1008,11913,10',
 '1008,10124,3',
 '1008,11105,1',
 '1008,12800,2',
 '1009,10075,4',
 '1009,9108,5',
 '1009,11168,9',
 '1009,9643,2',
 '1009,11800,5',
 '1009,13774,9',

#Pré-processamento dos dados

In [5]:
# Convertendo as strings
ratingsRDD2 = ratingsRDD.map(lambda l: l.split(',')).map(lambda l:(int(l[0]), int(l[1]), float(l[2])))
# Divide a linha por vírgula e transforma em (user, item, rating) como inteiros e float

In [7]:
# # Cria um DataFrame a partir do RDD, nomeando as colunas para facilitar análises e modelagem
ratingsDF = spark.createDataFrame(ratingsRDD2, ["user", "item", "rating"])

In [8]:
ratingsDF #Visualização do DataFrame criado

DataFrame[user: bigint, item: bigint, rating: double]

In [9]:
#Divisão entre treino e teste

trainDF, testDF = ratingsDF.randomSplit([0.8, 0.2], seed=42)

print("Training Data Count:", trainDF.count())
print("Testing Data Count:", testDF.count())

Training Data Count: 80137
Testing Data Count: 19957


#Algoritmo ALS - Sistema de Recomendação

In [12]:
#ALS com otimização de hiperparametros.

# Definindo os hiperparâmetros
paramGrid = ParamGridBuilder() \
    .addGrid(als.rank, [40,60]) \
    .addGrid(als.maxIter, [10, 15]) \
    .addGrid(als.regParam, [0.05, 0.1]) \
    .build()

# Definindo o evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

# Criando um CrossValidator
crossval = CrossValidator(estimator=als,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)  # Usando 3+ folds

# Executando a cross-validation, e escolhendo o melhor conjunto de parâmetros.
cvModel = crossval.fit(trainDF)

# Obtendo o melhor modelo gerado pelo CrossValidator
best_model = cvModel.bestModel

# Printando os melhores parâmetros encontrados
print("Best Rank:", best_model.rank)
print("Best MaxIter:", best_model._java_obj.parent().getMaxIter())
print("Best RegParam:", best_model._java_obj.parent().getRegParam())

# Fazendo previsões nos dados de teste usando o melhor modelo
predictions = best_model.transform(testDF)

# Avaliando o melhor modelo nos dados de teste
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))


Best Rank: 40
Best MaxIter: 10
Best RegParam: 0.05
Root-mean-square error = 3.6079307774264433


In [13]:
# Visualizando o Affinity Score pelo ID
modelo.userFactors.orderBy("id").collect()

[Row(id=1001, features=[0.7538014054298401, -0.9712556004524231, 0.06334768980741501, -0.28350260853767395, -0.27435579895973206, -1.101318597793579, 0.4696803689002991, 1.3318527936935425, -1.6169193983078003, 0.231626957654953]),
 Row(id=1002, features=[0.31338515877723694, -1.4678655862808228, -0.36701780557632446, 1.1840534210205078, -0.28835126757621765, -0.6284224987030029, -1.1685562133789062, 0.4588730037212372, 0.3842223882675171, 0.4673447608947754]),
 Row(id=1003, features=[-0.08989141136407852, -0.6711523532867432, 0.5640803575515747, 2.0997769832611084, 0.2439732849597931, -0.279715895652771, -2.088968276977539, 0.41177356243133545, -1.0765522718429565, 1.351747989654541]),
 Row(id=1004, features=[0.48306554555892944, -0.8590068817138672, 0.9793121218681335, -0.032308559864759445, 0.0025913547724485397, -0.3504028022289276, 1.7228708267211914, 0.6820260286331177, -1.985787034034729, 0.17766591906547546]),
 Row(id=1005, features=[-0.9631646275520325, -0.7358178496360779, 0.

In [16]:
# Criando um dataset de teste com usuários e items para rating
testeDF = spark.createDataFrame([(1001,9004),(1001,9005),(1001,9010)], ["user", "item"])

In [17]:
# Previsões
# Quanto maior o Affinity Score, maior a probabilidade do usuário aceitar uma recomendação
previsoes = (modelo.transform(testeDF).collect())
previsoes

[Row(user=1001, item=9004, prediction=9.12010383605957),
 Row(user=1001, item=9005, prediction=3.325920581817627),
 Row(user=1001, item=9010, prediction=5.5745344161987305)]

#Aplicação Web do Sistema de Recomendações usando Gradio

In [18]:
# Função para fazer a recomendação
def fazer_recomendacao(user_id):
    """
    Faz recomendações de itens para um determinado user_id.
    """
    # Cria um DataFrame de teste para o usuário especificado
    user_test_df = spSession.createDataFrame([(int(user_id), item) for item in ratingsDF.select('item').distinct().rdd.flatMap(lambda x: x).collect()], ["user", "item"])

    # Faz as previsões para o usuário
    previsoes = modelo.transform(user_test_df)

    # Ordena as previsões por rating previsto e seleciona os top N
    top_n_recomendacoes = previsoes.orderBy("prediction", ascending=False)

    # Retorna as recomendações como uma lista de tuplas (item, rating previsto)
    return [(row.item, row.prediction) for row in top_n_recomendacoes.collect()]

# Cria a interface Gradio
iface = gr.Interface(
    fn=fazer_recomendacao,
    inputs=gr.Textbox(label="Digite o ID do Usuário:"),
    outputs=gr.Dataframe(headers=["Item ID", "Rating Previsto"], label="Recomendações:"),
    title="Sistema de Recomendação com PySpark",
    description="Digite o ID de um usuário para obter recomendações de itens."
)

# Lança a interface
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c1a0bb7e4160e2a02f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
